In [4]:
from search import *  
from utils import *
import copy

class State:
    def __init__(self, row, col, roomMatrix):
        self.cleanerRow = row
        self.cleanerCol = col
        self.roomMatrix = roomMatrix
        

class VacuumProblem(Problem):

    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal


    def actions(self, state):
        possible_actions = ['up','left','down','right', 'suck']
        
        ultimaRiga = len(state.roomMatrix) - 1
        ultimaColonna = len(state.roomMatrix[0]) - 1

        #se si trova fuori dai bordi non ha azioni disponibili
        if state.cleanerRow < 0 or state.cleanerCol < 0 or state.cleanerRow > ultimaRiga or state.cleanerCol > ultimaColonna:
            return []
        
        #se sta nella parte interna della matrice, posso controllare senza preoccuparmi delle eccezioni
        elif state.cleanerRow in range(1, ultimaRiga) and state.cleanerCol in range(1, ultimaColonna):
            if state.roomMatrix[state.cleanerRow + 1][state.cleanerCol] == 'x':
                possible_actions.remove('down')
            
            if state.roomMatrix[state.cleanerRow - 1][state.cleanerCol] == 'x':
                possible_actions.remove('up')
                
            if state.roomMatrix[state.cleanerRow][state.cleanerCol + 1] == 'x':
                possible_actions.remove('right')   
                
            if state.roomMatrix[state.cleanerRow][state.cleanerCol - 1] == 'x':
                possible_actions.remove('left')
            
        #se sta in alto a sinistra, non può andare su e a sinistra, inoltre deve controllare sotto e a destra
        elif state.cleanerRow == 0 and state.cleanerCol == 0:
            possible_actions.remove('up')
            possible_actions.remove('left')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol + 1] == 'x':
                possible_actions.remove('right')
            if state.roomMatrix[state.cleanerRow + 1][state.cleanerCol] == 'x':
                possible_actions.remove('down')
                
        #se sta in alto a destra, non può andare su e a destra, inoltre deve controllare sotto e a sinitra
        elif state.cleanerRow == 0 and state.cleanerCol == ultimaColonna:
            possible_actions.remove('up')
            possible_actions.remove('right')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol - 1] == 'x':
                possible_actions.remove('left')
            if state.roomMatrix[state.cleanerRow + 1][state.cleanerCol] == 'x':
                possible_actions.remove('down')
                     
        #se sta in basso a sinistra, non può andare giu e a sinistra, inoltre deve controllare sopra e a destra
        elif state.cleanerRow == ultimaRiga and state.cleanerCol == 0:
            possible_actions.remove('down')
            possible_actions.remove('left')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol + 1] == 'x':
                possible_actions.remove('right')
            if state.roomMatrix[state.cleanerRow - 1][state.cleanerCol] == 'x':
                possible_actions.remove('up')
            
        #se sta in basso a destra, non può andare giu e a destra, inoltre deve controllare sopra e a sinistra
        elif state.cleanerRow == ultimaRiga and state.cleanerCol == ultimaColonna:
            possible_actions.remove('down')
            possible_actions.remove('right')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol - 1] == 'x':
                possible_actions.remove('left')
            if state.roomMatrix[state.cleanerRow - 1][state.cleanerCol] == 'x':
                possible_actions.remove('up')
             
        #se sta sul bordo e non sul''angolo, elimina la direzione che lo porta fuori
        #inoltre può controllare le altre tre direzioni senza preoccuparsi delle eccezioni
        elif state.cleanerRow == 0:
            possible_actions.remove('up')
            if state.roomMatrix[state.cleanerRow + 1][state.cleanerCol] == 'x':
                possible_actions.remove('down')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol + 1] == 'x':
                possible_actions.remove('right')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol - 1] == 'x':
                possible_actions.remove('left')
            
        elif state.cleanerRow == ultimaRiga:
            possible_actions.remove('down')
            if state.roomMatrix[state.cleanerRow - 1][state.cleanerCol] == 'x':
                possible_actions.remove('up')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol + 1] == 'x':
                possible_actions.remove('right')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol - 1] == 'x':
                possible_actions.remove('left')
            
        elif state.cleanerCol == 0:
            possible_actions.remove('left')
            if state.roomMatrix[state.cleanerRow][state.cleanerCol + 1] == 'x':
                possible_actions.remove('right')
            if state.roomMatrix[state.cleanerRow - 1][state.cleanerCol] == 'x':
                possible_actions.remove('up')
            if state.roomMatrix[state.cleanerRow + 1][state.cleanerCol] == 'x':
                possible_actions.remove('down')
                                
        elif state.cleanerCol == ultimaColonna:
            possible_actions.remove('right')  
            if state.roomMatrix[state.cleanerRow][state.cleanerCol - 1] == 'x':
                possible_actions.remove('left')
            if state.roomMatrix[state.cleanerRow - 1][state.cleanerCol] == 'x':
                possible_actions.remove('up')
            if state.roomMatrix[state.cleanerRow + 1][state.cleanerCol] == 'x':
                possible_actions.remove('down')
            
        #se è già pulito, non può aspirare ancora   
        lettera = state.roomMatrix[state.cleanerRow][state.cleanerCol]    
        if  lettera == 'c' or lettera == 'x' or lettera == 's' or lettera == 'f':
            possible_actions.remove('suck')

        return possible_actions


    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        if action == 'up':
            newRow = state.cleanerRow - 1
            newState = State(newRow, state.cleanerCol, state.roomMatrix) 
        
        elif action == 'down':
            newRow = state.cleanerRow + 1
            newState = State(newRow, state.cleanerCol, state.roomMatrix) 
        
        elif action == 'left':
            newCol = state.cleanerCol - 1
            newState = State(state.cleanerRow, newCol, state.roomMatrix) 
        
        elif action == 'right':
            newCol = state.cleanerCol + 1
            newState = State(state.cleanerRow, newCol, state.roomMatrix) 
        
        elif action == 'suck':
            newMatrix = copy.deepcopy(state.roomMatrix)
            newMatrix[state.cleanerRow][state.cleanerCol] = 'c'
            newState = State(state.cleanerRow, state.cleanerCol, newMatrix)
            
        return newState
       
        
    def goal_test(self, state):
        for roomRow in state.roomMatrix:
            for cell in roomRow:
                if cell == "d" or cell == 'v':
                    return False
                
        if state.roomMatrix[state.cleanerRow][state.cleanerCol] != 'f':
            return False
        
        return True
    

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        
        if action == 'suck':
            if state1.roomMatrix[state1.cleanerRow][state1.cleanerRow] == 'v':
                return c + 3
            if state1.roomMatrix[state1.cleanerRow][state1.cleanerRow] == 'd':
                return c + 2
        return c + 1
    
    
    def heuristic(self, state):
        h = 0
        for i in range(len(state.roomMatrix)):
            for j in range(len(state.roomMatrix[0])):
                if state.roomMatrix[i][j] == 'd' or state.roomMatrix[i][j] == 'v':
                    h += 1
        return h


    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [5]:
initial = State(2, 2, [['c', 'v','c'],['c', 'd', 'c'], ['s', 'd', 'v']])
provaPorbl = VacuumProblem(initial)

print('row ' + str(initial.cleanerRow) + ', col ' + str(initial.cleanerCol))
print(initial.roomMatrix)
print(provaPorbl.actions(initial))
a = provaPorbl.heuristic(initial)
print(a)

print()

newState = provaPorbl.result(provaPorbl.initial,'suck')
print('row ' + str(newState.cleanerRow) + ', col ' + str(newState.cleanerCol))
print(newState.roomMatrix)
print(provaPorbl.actions(newState))



row 2, col 2
[['c', 'v', 'c'], ['c', 'd', 'c'], ['s', 'd', 'v']]
['up', 'left', 'suck']
4

row 2, col 2
[['c', 'v', 'c'], ['c', 'd', 'c'], ['s', 'd', 'c']]
['up', 'left']


In [27]:
class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state. Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node. Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1


    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]


    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node


    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]


    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state


    def __hash__(self):
        # We use the hash value of the state
        # stored in the node instead of the node
        # object itself to quickly search a node
        # with the same state in a Hash Table
        return hash(self.state)


    def __repr__(self):
        return "<Node {}>".format(self.state)


    def __lt__(self, node):
        return f(self) < f(node)

In [28]:
fisrtState = State(0, 0, [['d', 'c'],['c', 'c'], ['x', 's']])
provaPorbl = VacuumProblem(fisrtState)
firstNode = Node(provaPorbl.initial)

children = firstNode.expand(provaPorbl)
for n in children:
    print(n.action)

down
right
suck


In [29]:
def f(node, problem):
    return node.path_cost + problem.h(node)

fisrtState = State(0, 0, [['v', 'c'],['c', 'c'], ['x', 's']])
provaPorbl = VacuumProblem(fisrtState)
firstNode = Node(provaPorbl.initial)

nextNode = firstNode.child_node(provaPorbl, 'down')
print(f(nextNode, provaPorbl))

7
